In [1]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor

# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)



INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2020.09.4 jupyter extensions


In [7]:
outdir = '/data/chodera/zhangi/perses_benchmark/neq/14/11/'
resid = '439'
old_aa_name = 'ASN'
new_aa_nname = 'LYS'
sim_number = 1

if sim_number == 1:
    phase = 'apo'
    name = old_aa_name
    state = 0
elif sim_number == 2:
    phase = 'apo'
    name = new_aa_name
    state = 1
elif sim_number == 3:
    phase = 'complex'
    name = old_aa_name
    state = 0
elif sim_number == 4:
    phase = 'complex'
    name = new_aa_name
    state = 1

length = 1
move_length = 3
timestep = 4
radius = 0.3

In [8]:
# Load rhtf 
i = os.path.basename(os.path.dirname(outdir))
path = os.path.join(outdir, f"{i}_{phase}_{state}.pickle")
_logger.info(f"path: {path}")
htf = pickle.load(open(path, "rb" ))

# Build REST factory
_logger.info("Generating REST factory")
_logger.info(f"radius:{radius} nm")
# query_indices = [atom.index for atom in list(htf.hybrid_topology.residues)[int(args.resid)].atoms]
for res in htf.hybrid_topology.residues:
    if res.resSeq == int(resid) and res.chain.index == 0:
        mutated_res = res
query_indices = [atom.index for atom in mutated_res.atoms]
_logger.info(f"query indices {query_indices}")
traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
_logger.info(f"rest atoms {rest_atoms}")
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=rest_atoms)

_logger.info("Generating REST states")
# Get REST system
REST_system = factory.REST_system

# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 1200 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTState.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

context_cache = cache.ContextCache()

# Create thermodynamics states
sampler_state =  SamplerState(htf.hybrid_positions, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state, max_iterations=0)
    sampler_state_list.append(copy.deepcopy(sampler_state))



INFO:root:path: /data/chodera/zhangi/perses_benchmark/neq/14/11/11_apo_0.pickle
INFO:root:Generating REST factory
INFO:root:radius:0.3 nm
INFO:root:query indices [1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1642, 1643, 1644, 1645, 1646, 58633, 58634, 58635, 58636, 58637, 58638, 58639, 58640, 58641, 58642, 58643, 58644, 58645]
<ipython-input-8-181f135c1d69>:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
INFO:root:rest atoms [1615, 16

In [11]:
# Set up sampler
_logger.setLevel(logging.DEBUG)
_logger.info("About to start repex")
print(f"move steps: {int((move_length*1000)/timestep)}")
print(f"timestep: {timestep} fs")
move = mcmc.LangevinSplittingDynamicsMove(timestep=timestep*unit.femtoseconds, n_steps=int((move_length*1000)/timestep))
simulation = multistate.ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=20)

# Run t-repex
reporter_file = os.path.join(outdir, f"test_normal.nc")
reporter = multistate.MultiStateReporter(reporter_file, checkpoint_interval=1)
simulation.create(thermodynamic_states=thermodynamic_state_list,
                  sampler_states=sampler_state_list,
                  storage=reporter)
simulation.run()


INFO:root:About to start repex
DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as /data/chodera/zhangi/perses_benchmark/neq/14/11/test_normal_checkpoint.nc
DEBUG:mpiplus.mpiplus:Cannot find MPI environment. MPI disabled.
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2abd7bf80850>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2abdc1b3e430>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2abdc1afcd30>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2abdc1b00040>


move steps: 750
timestep: 4 fs
Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys.,

DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  779767B | 761.491KB | 0.744MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    2.367s
DEBUG:openmmtools.multistate.multistatesampler:Storing general ReplicaExchange options...
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2abdc1b00160>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2abdc1b00430>
DEBUG:openmmtools.utils:Storing sampler states took    0.021s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.421s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.637s
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2abd7bf80850>>
DEBUG:openmmtools.multistate.multistat

DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 4/20
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2abdc1b3e0d0>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.002s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 6130/41472 attempted swaps (14.8%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took   17.172s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Com

DEBUG:openmmtools.multistate.multistatesampler:*** ONLINE analysis free energies:
DEBUG:openmmtools.multistate.multistatesampler:        -0.0    20.4    20.5    20.5    20.5    20.5    20.5    20.5    20.5    20.5    20.5    20.5
DEBUG:openmmtools.utils:Computing online free energy estimate took    0.009s
DEBUG:openmmtools.multistate.multistatesampler:Iteration took 18.584s.
DEBUG:openmmtools.multistate.multistatesampler:Estimated completion in 0:04:01.156804, at Wed Mar 24 18:40:09 2021 (consuming total wall clock time 0:06:11.010468).
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 8/20
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2abdc1b3e0d0>
DEBUG:openmmtools.multistate

DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2abdc1b00430>
DEBUG:openmmtools.utils:Storing sampler states took    0.023s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.433s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2abdc1b00f70>
DEBUG:openmmtools.multistate.multistatesampler:*** ONLINE analysis free energies:
DEBUG:openmmtools.multistate.multistatesampler:        -0.0    24.9    24.9    24.9    25.0    25.0    25.0    25.0    25.0    25.0    24.9    24.9
DEBUG:openmmtools.utils:Computing online free energy estimate took    0.010s
DEBUG:openmmtools.multistate.multistatesampler:Iteration took 19.713s.
DEBUG:openmmtools.multistate.multistatesampler:Estimated completion in 0:02:49.563966, at Wed Mar 24 18:40:14 2021 (co

DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took   19.722s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.354s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2abdc1b00160>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2abdc1b00430>
DEBUG:openmmtools.utils:Storing sampler states took    0.020s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.429s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2abd

DEBUG:openmmtools.multistate.multistatesampler:Iteration 19/20
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2abdc1b3e0d0>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.002s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 6180/41472 attempted swaps (14.9%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took   20.811s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.378s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration 

In [12]:
simulation._sampler_states

In [15]:
simulation._thermodynamic_states[0].temperature

Quantity(value=298.0, unit=kelvin)

In [17]:
from perses.analysis.utils import open_netcdf
nc = open_netcdf(os.path.join(outdir, f"test_normal.nc"))
nc_checkpoint = open_netcdf(os.path.join(outdir, f"test_normal_checkpoint.nc"))
checkpoint_interval = nc_checkpoint.CheckpointInterval
all_positions = nc_checkpoint.variables['positions']
n_iter, n_replicas, n_atoms, _ = np.shape(all_positions)



In [ ]:
from tqdm import tqdm
all_pos_new = np.zeros(shape=(n_iter, new_top.n_atoms, 3))
all_pos_old = np.zeros(shape=(n_iter, old_top.n_atoms, 3))
all_pos_hybrid = np.zeros(shape=(n_iter, n_atoms, 3))
for iteration in tqdm(range(n_iter)):
    replica_id = np.where(nc.variables['states'][iteration*checkpoint_interval] == 0)[0]
    pos = all_positions[iteration,replica_id,:,:][0] *unit.nanometers
    all_pos_new[iteration] = new_positions(htf, pos).value_in_unit_system(unit.md_unit_system) # Get new positions only
    all_pos_hybrid[iteration] = pos # Get hybrid positions
    all_pos_old[iteration] = old_positions(htf, pos).value_in_unit_system(unit.md_unit_system)


In [21]:
nc.variables['states']

<class 'netCDF4._netCDF4.Variable'>
int32 states(iteration, replica)
    units: none
    long_name: states[iteration][replica] is the thermodynamic state index (0..n_states-1) of replica 'replica' of iteration 'iteration'.
unlimited dimensions: iteration
current shape = (21, 12)
filling on, default _FillValue of -2147483647 used

In [20]:
for i in range(20):
    print(nc.variables['states'][i])

[ 0  1  2  3  4  5  6  7  8  9 10 11]
[ 0  1  4  2  6  5  3  8  7 10 11  9]
[ 0  1  5  2  6  4  3  9  7 10 11  8]
[ 0  1  5  3  6  2  4  9  7 10 11  8]
[ 0  1  5  2  9  4  3  8  6 11 10  7]
[ 0  2  6  1  9  4  3  8  7 10 11  5]
[ 0  2  6  1 10  4  3  8  7 11  9  5]
[ 0  2  8  1 10  3  4  9  6 11  7  5]
[ 0  2  9  1 10  3  5  8  6 11  7  4]
[ 0  2  8  1 10  4  6  9  5 11  7  3]
[ 0  2  8  1 11  4  6  7  5  9 10  3]
[ 0  1  8  2 11  4  5  9  6  7 10  3]
[ 1  0  9  2 11  3  7  8  5  6 10  4]
[ 3  0  9  1 11  2  6  8  5  7 10  4]
[ 3  0  8  1 11  2  5 10  6  7  9  4]
[ 3  0 10  1 11  2  4  7  9  5  8  6]
[ 2  1 10  0 11  4  3  7  9  6  8  5]
[ 1  2  9  0 11  5  3  8 10  4  7  6]
[ 2  1  9  0 10  7  3  8 11  4  5  6]
[ 2  0  8  1 11  6  3  9 10  4  5  7]


In [25]:
nc.variables['last_iteration']

<class 'netCDF4._netCDF4.Variable'>
int64 last_iteration(scalar)
unlimited dimensions: 
current shape = (1,)
filling on, default _FillValue of -9223372036854775806 used

In [47]:
from openmmtools.multistate import ReplicaExchangeSampler
import time

In [48]:
import mpiplus
class ReplicaExchangeSampler2(ReplicaExchangeSampler):
    @mpiplus.on_single_node(rank=0, broadcast_result=False, sync_nodes=False)
    @mpiplus.delayed_termination
    def _report_iteration_items(self):
        """
        Sub-function of :func:`_report_iteration` which handles all the actual individual item reporting in a
        sub-class friendly way. The final actions of writing timestamp, last-good-iteration, and syncing
        should be left to the :func:`_report_iteration` and subclasses should extend this function instead
        """
        replica_id = np.where(self._replica_thermodynamic_states == 0)[0][0]
        print("REPLICA THERMOSTATES ", self._replica_thermodynamic_states, type(self._replica_thermodynamic_states))
        print("REPLICA ID ", replica_id, type(replica_id))
        initial_time = time.time()
        self._reporter.write_sampler_states([self._sampler_states[replica_id]], self._iteration)
        elapsed_time = (time.time() - initial_time) * unit.seconds
        print("ELAPSED TIME FROM WRITE_SAMPLER_STATE: ", elapsed_time)
        
        initial_time = time.time()
        n_particles = self._sampler_states[replica_id].n_particles
        n_replicas = 1
        positions = np.zeros([n_replicas, n_particles, 3])
        x = self._sampler_states[replica_id].positions / unit.nanometers
        positions[0, :, :] = x[:, :]
        np.save(os.path.join(outdir, f"{self._iteration}.npy"), positions)
        elapsed_time = (time.time() - initial_time) * unit.seconds
        print("ELAPSED TIME FROM SAVE NPY: ", elapsed_time)
        
        initial_time = time.time()
        n_particles = self._sampler_states[replica_id].n_particles
        n_replicas = 1
        positions = np.zeros([n_replicas, n_particles, 3])
        x = self._sampler_states[replica_id].positions / unit.nanometers
        positions[0, :, :] = x[:, :]
        np.savez_compressed(os.path.join(outdir, f"{self._iteration}.npz"), positions)
        elapsed_time = (time.time() - initial_time) * unit.seconds
        print("ELAPSED TIME FROM SAVE NPZ: ", elapsed_time)
        
        self._reporter.write_replica_thermodynamic_states(self._replica_thermodynamic_states, self._iteration)
        self._reporter.write_mcmc_moves(self._mcmc_moves)  # MCMCMoves can store internal statistics.
        self._reporter.write_energies(self._energy_thermodynamic_states, self._neighborhoods, self._energy_unsampled_states,
                                      self._iteration)
        self._reporter.write_mixing_statistics(self._n_accepted_matrix, self._n_proposed_matrix, self._iteration)

In [49]:
# Set up sampler
_logger.setLevel(logging.DEBUG)
_logger.info("About to start repex")
print(f"move steps: {int((move_length*1000)/timestep)}")
print(f"timestep: {timestep} fs")
move = mcmc.LangevinSplittingDynamicsMove(timestep=timestep*unit.femtoseconds, n_steps=int((move_length*1000)/timestep))
simulation = ReplicaExchangeSampler2(mcmc_moves=move, number_of_iterations=20)

# Run t-repex
reporter_file = os.path.join(outdir, f"test_replica_0_only.nc")
reporter = multistate.MultiStateReporter(reporter_file, checkpoint_interval=1)
simulation.create(thermodynamic_states=thermodynamic_state_list,
                  sampler_states=sampler_state_list,
                  storage=reporter)
simulation.run()


INFO:root:About to start repex
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.close of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2abe1f0ede50>>
DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as /data/chodera/zhangi/perses_benchmark/neq/14/11/test_replica_0_only_checkpoint.nc
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2abdfbce7c10>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2abdc1b3e430>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2abdc1afcd30>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2abdc1b00040>


move steps: 750
timestep: 4 fs
Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys.,

DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  779767B | 761.491KB | 0.744MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    2.331s
DEBUG:openmmtools.multistate.multistatesampler:Storing general ReplicaExchange options...
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2abdc1b00160>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler2._report_iteration_items at 0x2abdfbd3af70>
DEBUG:openmmtools.utils:Storing sampler states took    0.006s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.184s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.


REPLICA THERMOSTATES  [ 0  1  2  3  4  5  6  7  8  9 10 11] <class 'numpy.ndarray'>
REPLICA ID  0 <class 'numpy.int64'>
ELAPSED TIME FROM WRITE_SAMPLER_STATE:  0.00812673568725586 s
ELAPSED TIME FROM SAVE NPY:  0.008871078491210938 s
ELAPSED TIME FROM SAVE NPZ:  0.09959697723388672 s


DEBUG:openmmtools.utils:Computing energy matrix took    0.372s
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2abdfbce7c10>>
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 1/20
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2abdc1b3e0d0>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.002s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 13302/41472 attempted swaps (32.1%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propa

...

REPLICA THERMOSTATES  [ 0  1  6  4  3  5 10  2  8 11  7  9] <class 'numpy.ndarray'>
REPLICA ID  0 <class 'numpy.int64'>
ELAPSED TIME FROM WRITE_SAMPLER_STATE:  0.005605220794677734 s
ELAPSED TIME FROM SAVE NPY:  0.006827116012573242 s
ELAPSED TIME FROM SAVE NPZ:  0.10933613777160645 s


DEBUG:openmmtools.multistate.multistatesampler:Iteration 19/20
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2abdc1b3e0d0>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.002s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 5968/41472 attempted swaps (14.4%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took   20.665s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.391s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration 

REPLICA THERMOSTATES  [ 0  1  6  4  2  5 10  3  8 11  7  9] <class 'numpy.ndarray'>
REPLICA ID  0 <class 'numpy.int64'>
ELAPSED TIME FROM WRITE_SAMPLER_STATE:  0.005522727966308594 s
ELAPSED TIME FROM SAVE NPY:  0.0064792633056640625 s
ELAPSED TIME FROM SAVE NPZ:  0.11040019989013672 s


DEBUG:openmmtools.multistate.multistatesampler:Iteration 20/20
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2abdc1b3e0d0>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.002s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 6508/41472 attempted swaps (15.7%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took   19.432s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.370s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration 

REPLICA THERMOSTATES  [ 0  1  5  3  2  6  8  4  7 11  9 10] <class 'numpy.ndarray'>
REPLICA ID  0 <class 'numpy.int64'>
ELAPSED TIME FROM WRITE_SAMPLER_STATE:  0.005652189254760742 s
ELAPSED TIME FROM SAVE NPY:  0.0073337554931640625 s
ELAPSED TIME FROM SAVE NPZ:  0.10961675643920898 s
